# 📊 Documentation: Benchmarking on All Layouts

The library provides functionality to **benchmark wildfire scenarios across multiple layouts** in a dataset. This is useful for evaluating how different **drone routing** and **sensor placement** strategies perform under varied spatial conditions.

---

## 🛠️ Function: `run_benchmark_scenarii_parallel(...)`

Benchmarks all wildfire scenarios found in a specified directory, applying the given strategies to each layout. The function supports **parallel execution** to speed up the benchmarking process.

### **Function Signature**

```python
run_benchmark_scenarii_parallel(
    input_dir,
    sensor_placement_strategy,
    drone_routing_strategy,
    custom_initialization_parameters_function,
    custom_step_parameters_function,
    file_format,
    simulation_parameters
)


## 📥 Parameters

### `input_dir` *(str)*
Path to the directory containing scenario files (in `.npy` or `.jpg` format).

### `sensor_placement_strategy`
Function or strategy object for placing ground sensors and charging stations across each layout.

### `drone_routing_strategy`
Function or strategy object for defining drone movement and behavior.

### `custom_initialization_parameters_function`
A user-defined function that sets up initialization parameters such as drone count, start positions, or initial battery levels.

### `custom_step_parameters_function`
A function that configures or updates simulation behavior and parameters at each timestep.

### `file_format` *(str)*
Format of the scenario files in the dataset. Must be either `"npy"` or `"jpg"`.

### `simulation_parameters` *(dict)*
A dictionary containing simulation-specific settings. Example keys include:
- `drone_speed`
- `sensor_coverage_radius`
- `battery_capacity`
- `max_timesteps`
- `communication_range`

---

## 📤 Returns

A dictionary containing aggregated benchmark metrics across all layouts. Results typically include:

- ⏱️ **Total and average execution time**
- 🔥 **Average and total fire size**
- 🚁 **Drone coverage performance per layout**
- 📡 **Sensor detection rates**
- 🔋 **Energy/resource usage statistics**
- ✅ **Success rates across different scenarios**


In [ ]:
# Import necessary modules
import os
import numpy as np
import sys

module_path = os.path.abspath("..") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Added {module_path} to sys.path")


from benchmark import run_benchmark_scenarii_parallel
from wrappers import wrap_log_sensor_strategy, wrap_log_drone_strategy
from Strategy import RandomSensorPlacementStrategy, DroneRoutingUniformMaxCoverageResetStatic
from new_clustering import get_wrapped_clustering_strategy

# Define paths and parameters
input_dir = "../MinimalDataset/"
file_format = "npy"

# Define simulation parameters
simulation_parameters = {
    "max_battery_distance": -1,
    "max_battery_time": 1,
    "n_drones": 3,
    "n_ground_stations": 4,
    "n_charging_stations": 2,
    "drone_speed_m_per_min": 600,
    "coverage_radius_m": 300,
    "cell_size_m": 30,
    "transmission_range": 50000,
}

# Define strategies
sensor_strategy = wrap_log_sensor_strategy(RandomSensorPlacementStrategy)
drone_strategy = wrap_log_drone_strategy(get_wrapped_clustering_strategy(DroneRoutingUniformMaxCoverageResetStatic))

# Define custom initialization parameters
def custom_initialization_parameters_function(input_dir: str):
    return {
        "burnmap_filename": f"{input_dir.strip('/')}/static_risk.npy",
        "reevaluation_step": 5,
        "optimization_horizon": 10,
        "regularization_param": 1,
    }

# Run benchmark across all layouts
metrics = run_benchmark_scenarii_parallel(
    input_dir,
    sensor_strategy,
    drone_strategy,
    custom_initialization_parameters_function,
    lambda: {},  # No custom step parameters
    file_format=file_format,
    simulation_parameters=simulation_parameters,
)

# Print aggregated metrics
print("Aggregated Metrics Across All Layouts:")
print(metrics)

# Expected Output
A dictionary containing aggregated metrics across all layouts, such as:

avg_time_to_detection: Average time taken to detect the wildfire.

device_percentages: Percentage usage of each device (e.g., drones, sensors).

avg_execution_time: Average execution time across all layouts.

avg_fire_size: Average size of the fire across all layouts.

avg_map_explored: Average percentage of the map explored by drones.